In [1]:
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
train_messy = "images/train/messy"
train_clean= "images/train/clean"
test_messy= "images/val/messy"
test_clean= "images/val/clean"
image_size = 128

In [3]:
def train_data():
    train_data_messy = [] 
    train_data_clean=[]
    for image1 in tqdm(os.listdir(train_messy)): 
        path = os.path.join(train_messy, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        train_data_messy.append(img1) 
    for image2 in tqdm(os.listdir(train_clean)): 
        path = os.path.join(train_clean, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        train_data_clean.append(img2) 
    
    train_data= np.concatenate((np.asarray(train_data_messy),np.asarray(train_data_clean)),axis=0)
    return train_data 

In [4]:
def test_data():
    test_data_messy = [] 
    test_data_clean=[]
    for image1 in tqdm(os.listdir(test_messy)): 
        path = os.path.join(test_messy, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        test_data_messy.append(img1) 
    for image2 in tqdm(os.listdir(test_clean)): 
        if image2[0] == ".":
            continue
        path = os.path.join(test_clean, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        test_data_clean.append(img2) 
    
    test_data= np.concatenate((np.asarray(test_data_messy),np.asarray(test_data_clean)),axis=0) 
    return test_data 

In [5]:
train_data = train_data() 
test_data = test_data()

100%|██████████| 11/11 [00:00<00:00, 168.62it/s]


In [6]:
x_data=np.concatenate((train_data,test_data),axis=0)
x_data = (x_data.astype(np.float32) - 127.5) / 127.5

In [7]:
z1 = np.zeros(96)
o1 = np.ones(96)
Y_train = np.concatenate((o1, z1), axis=0)
z = np.zeros(10)
o = np.ones(10)
Y_test = np.concatenate((o, z), axis=0)

In [8]:
y_data=np.concatenate((Y_train,Y_test),axis=0).reshape(x_data.shape[0],1)

In [9]:
x_data.shape,y_data.shape

((212, 128, 128), (212, 1))

In [10]:
X = x_data.reshape(x_data.shape[0],x_data.shape[1]*x_data.shape[2])
y = y_data
X.shape,y.shape

((212, 16384), (212, 1))

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [13]:
param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel type: linear or radial basis function
    'gamma': [0.1, 1, 'scale'],  # Kernel coefficient (scale for 1 / (n_features * X.var()))
}
svc = SVC()


In [14]:
grid_search = GridSearchCV(svc, param_grid, cv=5)  
grid_search.fit(X_train, y_train.ravel())  


KeyboardInterrupt: 

In [ ]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_params,best_model

In [50]:
best_model.score(X_test,y_test)

0.7285714285714285

In [77]:
class Dense_Layer:
    def __init__(self,ninputs,nneurons,l1_w=0,l1_b=0,l2_w=0,l2_b=0):
        # Initialising weights and biases
        self.weights = 0.001 * np.random.randn(ninputs,nneurons)
        self.biases = np.zeros((1,nneurons))
        # Regularization
        self.l1_w = l1_w
        self.l1_b = l1_b
        self.l2_w = l2_w
        self.l2_b = l2_b
        
    # Forward Propagation    
    def forward(self,inputs,training):
        self.output = np.dot(inputs,self.weights) + self.biases
        self.inputs = inputs
    
    # Backpropagation
    def backward(self,dvalues):
        self.dinputs = np.dot(dvalues,self.weights.T)
        self.dweights = np.dot(self.inputs.T,dvalues)
        self.dbiases = np.sum(dvalues,axis=0,keepdims=True)
        
        if self.l1_w > 0:
            dl1w = np.ones_like(self.weights)
            dl1w[self.weights < 0] = -1
            self.dweights += dl1w
        if self.l1_b > 0:
            dl1b = np.ones_like(self.biases)
            dl1b[self.biases < 0] = -1
            self.dbiases += sl1b
        if self.l2_w > 0:
            self.dweights += self.weights * 2 * self.l2_w
        if self.l2_b > 0:
            self.dbiases += self.biases * 2 * self.l2_b
            

class Dropout_layer:
    def __init__(self,drop_rate=0):
        self.drop_rate = 1 - drop_rate
    def forward(self,inputs,training):
        if not training:
            self.output = input.copy()
            return
        self.dropmask = np.random.binomial(1,self.drop_rate,size =inputs.shape)/(self.drop_rate)
        self.output = inputs * self.dropmask
    def backward(self,dvalues):
        self.dinputs = dvalues * self.dropmask
        
class Activation_Relu:
    def forward(self,inputs,training):
        self.output = np.maximum(0,inputs)
        self.inputs = inputs
    
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
        
class Activation_Sigmoid:
    def forward(self,inputs,training):
        self.inputs= inputs
        self.outputs = 1/(1+np.exp(-inputs))
    def backward(self,dvalues):
        self.dinputs = dvalues * self.outputs * (1 - self.outputs)
        
class Loss_BinaryCrossentropy():
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        # Return loss
        return data_loss

    def regularization_loss(self):
        
        loss = 0
        for layer in self.trainable_layers:
            if layer.l1_w > 0:
                loss += layer.l1_w * np.sum(np.abs(layer.weights))
            if layer.l1_b > 0:
                loss += layer.l1_b * np.sum(np.abs(layer.biases))
            if layer.l2_w > 0:
                loss += layer.l2_w * np.sum(layer.weights * layer.weights)
            if layer.l2_b > 0:
                loss += layer.l2_b * np.sum(layer.biases * layer.biases)
        
        return loss
    
    def remember_trainable_layers(self, trainable_layers):
        self.trainable_layers = trainable_layers
    
    # Forward pass
    def forward(self, y_pred, y_true):
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate sample-wise loss
        sample_losses = -(y_true * np.log(y_pred_clipped) +
        (1 - y_true) * np.log(1 - y_pred_clipped))
        sample_losses = np.mean(sample_losses, axis=-1)
        # Return losses
        return sample_losses
        # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of outputs in every sample
        # We'll use the first sample to count them
        outputs = len(dvalues[0])
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        clipped_dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)
        # Calculate gradient
        self.dinputs = -(y_true / clipped_dvalues -
        (1 - y_true) / (1 - clipped_dvalues)) / outputs
        # Normalize gradient
        self.dinputs = self.dinputs / samples
        
class Activation_Softmax:
    def forward(self,inputs,y_true):
        expvals = np.exp(inputs - np.max(inputs, axis=1,
        keepdims=True) )
        self.output = expvals/np.sum(expvals,axis=1,keepdims=True)
        
    def predictions(self, outputs):
        return np.argmax(outputs, axis=1)
    

class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
            range(samples),
            y_true
            ]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
            y_pred_clipped * y_true,
            axis=1
            )
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def calculate_accumulated(self,*,regularization=False):
        data_loss = self.accumulated_sum / self.accumulated_count
        if not regularization:
            return data_loss
        return data_loss,self.regularization_loss()
    
    def calculate(self, output, y, *, regularization=False):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        
        self.accumulated_sum += np.sum(sample_losses)
        self.accumulated_count += len(sample_losses)

        # If just data loss - return it
        if not regularization:
            return data_loss
        # Return the data and regularization losses
        return data_loss, self.regularization_loss()
        
        return np.mean(-np.log(y_pred[range(len(y_pred)),y_true]))
    def regularization_loss(self):
        
        loss = 0
        for layer in self.trainable_layers:
            if layer.l1_w > 0:
                loss += layer.l1_w * np.sum(np.abs(layer.weights))
            if layer.l1_b > 0:
                loss += layer.l1_b * np.sum(np.abs(layer.biases))
            if layer.l2_w > 0:
                loss += layer.l2_w * np.sum(layer.weights * layer.weights)
            if layer.l2_b > 0:
                loss += layer.l2_b * np.sum(layer.biases * layer.biases)
        
        return loss
    
    def remember_trainable_layers(self, trainable_layers):
        self.trainable_layers = trainable_layers
        
    def new_pass(self):
        self.accumulated_sum = 0
        self.accumulated_count = 0
        
        
class Activation_softmax_cross_entropy:
#     def __init__(self):
#         self.activation = Activation_Softmax()
#         self.lossfunc = CrossEntropyLoss()
    
#     def forward(self,inputs,y_true):
#         self.activation.forward(inputs,y_true)
#         self.output = self.activation.output
#         return self.lossfunc.calculate(self.output,y_true)
    
    def backward(self,dvalues,y_true):
        samples = len(y_true)
        
        # Turning one hot encoded arrays to sparse vectors
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true,axis=1)
        
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples),y_true] -= 1
        
        self.dinputs/=samples
        
        
class Adam_Optimizer:
    def __init__(self,lr=0.001,decay_rate=0,epsilon= 1e-7,beta1=0.9,beta2=0.999):
        self.initiallr = lr
        self.currentlr = lr
        self.decay_rate = decay_rate
        self.iterations = 0
        self.epsilon = epsilon
        self.beta1 = beta1
        self.beta2 = beta2
    
    def pre_update(self):
        self.currentlr = self.initiallr * (1/(1+(self.decay_rate * self.iterations)))

    def update_params(self,layer):
        
        if not hasattr(layer,'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
            layer.weight_momentum = np.zeros_like(layer.weights)
            layer.bias_momentum = np.zeros_like(layer.biases)
            
        layer.weight_momentum = self.beta1 * layer.weight_momentum + (1-self.beta1) * layer.dweights
        layer.bias_momentum = self.beta1 * layer.bias_momentum + (1 - self.beta1) * layer.dbiases
        
        layer.weight_momentum_prime = layer.weight_momentum / (1 - self.beta1 ** (self.iterations + 1))
        layer.bias_momentum_prime = layer.bias_momentum / (1 - self.beta1 ** (self.iterations + 1))
        
        layer.weight_cache = layer.weight_cache * self.beta2 + (1-self.beta2) * (layer.dweights ** 2)
        layer.bias_cache = layer.bias_cache * self.beta2 + (1-self.beta2) * (layer.dbiases ** 2)
        
        layer.weight_cache_prime = layer.weight_cache / (1 - self.beta2 ** (self.iterations + 1))
        layer.bias_cache_prime = layer.bias_cache / (1 - self.beta2 ** (self.iterations + 1))

        layer.weights += - self.currentlr * layer.weight_momentum_prime  / (np.sqrt(layer.weight_cache_prime) + self.epsilon)
        layer.biases += -self.currentlr * layer.bias_momentum_prime / (np.sqrt(layer.bias_cache_prime) + self.epsilon)
        
    def post_update(self):
        self.iterations += 1
        
        
class Model:
    def __init__(self):
        self.layers = []
        self.softmax_classifier_output = None
    
    def add(self,layer):
        self.layers.append(layer)
        
    def set(self,*,loss,optimizer,accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuracy = accuracy
    
    def train(self,X,y,*,epochs,print_every,validation_data = None,batch_size = None):
        self.accuracy.init(y)
        
        train_steps = 1
        
        if validation_data is not None:
            validation_steps = 1
            X_val,y_val = validation_data
        
        if batch_size is not None:
            train_steps = len(X) // batch_size
            
            if train_steps * batch_size < len(X):
                train_steps += 1
                
            if validation_data is not None:
                validation_steps = len(X_val) // batch_size
                
                if validation_steps * batch_size < len(X_val):
                    validation_steps += 1
                
        
        for epoch in range(1,epochs+1):
            
            print("Epoch : ",epoch)
            
            self.loss.new_pass()
            self.accuracy.new_pass()
            
            for step in range(train_steps):
                
                if batch_size is None:
                    batch_X = X
                    batch_y = y
                else:
                    batch_X = X[step * batch_size : (step+1) * batch_size]
                    batch_y = y[step * batch_size : (step+1) * batch_size]

                    
                output = self.forward(batch_X,training = True)
                data_loss,regularization_loss = self.loss.calculate(output,batch_y,regularization=True)
                loss = data_loss + regularization_loss

                predictions = self.output_activation.predictions(output)

                accuracy = self.accuracy.calculate(predictions,batch_y)

                self.backward(output,batch_y)

                self.optimizer.pre_update()
                for layer in self.trainable_layers:
                    self.optimizer.update_params(layer)
                self.optimizer.post_update

                if not step % print_every or step == train_steps - 1:
                    print(f'iteration: {step}, ' +
                    f'acc: {accuracy:.3f}, ' +
                    f'loss: {loss:.3f} (' +
                    f'data_loss: {data_loss:.3f}, ' +
                    f'reg_loss: {regularization_loss:.3f}), ' +
                    f'lr: {self.optimizer.currentlr}')
            
            epoch_accuracy = self.accuracy.calculate_accumulated()
            epoch_data_loss, epoch_regularization_loss = self.loss.calculate_accumulated(regularization=True)
            epoch_loss = epoch_data_loss + epoch_regularization_loss
            
            print(f'training, ' +
                    f'acc: {epoch_accuracy:.3f}, ' +
                    f'loss: {epoch_loss:.3f} (' +
                    f'data_loss: {epoch_data_loss:.3f}, ' +
                    f'reg_loss: {epoch_regularization_loss:.3f}), ' +
                    f'lr: {self.optimizer.currentlr}')

            
            
        if validation_data is not None:
            
            self.accuracy.new_pass()
            self.loss.new_pass()
            
            for step in range(validation_steps):
                if batch_size is None:
                    batch_X = X_val
                    batch_y = y_val
                else:
                    batch_X = X_val[step * batch_size : (step + 1) * batch_size]
                    batch_y = y_val[step * batch_size : (step + 1) * batch_size]
                    
            
            output = self.forward(batch_X,training=False)
            loss = self.loss.calculate(output, batch_y)
            predictions = self.output_activation.predictions(
            output)
            accuracy = self.accuracy.calculate(predictions, batch_y)
            validation_accuracy = self.accuracy.calculate_accumulated()
            validation_loss = self.loss.calculate_accumulated()
            print(f'validation, ' +
            f'acc: {validation_accuracy:.3f}, ' +
            f'loss: {validation_loss:.3f}')

        
    def finalize(self):
        self.input_layer = Input_Layer()
        self.trainable_layers = []
        nlayers = len(self.layers)
        
        for i in range(nlayers):
            if i==0 :
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
            elif i < nlayers - 1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                self.output_activation = self.layers[i]
            
            if hasattr(self.layers[i],"weights"):
                self.trainable_layers.append(self.layers[i])
        self.loss.remember_trainable_layers(self.trainable_layers)   
        
        if isinstance(self.layers[-1], Activation_Softmax) and \
        isinstance(self.loss, CrossEntropyLoss):
            self.softmax_classifier_output = \
            Activation_softmax_cross_entropy()

            
    def forward(self,X,training):
        self.input_layer.forward(X)
        for layer in self.layers:
            layer.forward(layer.prev.output,training)
        return layer.output
    def backward(self,output,y):
        
        if self.softmax_classifier_output is not None:
            self.softmax_classifier_output.backward(output, y)
            self.layers[-1].dinputs = \
            self.softmax_classifier_output.dinputs
            for layer in reversed(self.layers[:-1]):
                layer.backward(layer.next.dinputs)
            return
        
        
        self.loss.backward(output,y)
        
        for layer in reversed(self.layers):
            layer.backward(layer.next.dinputs)
        
class Input_Layer:
    def forward(self,inputs):
        self.output = inputs
        
class Accuracy:
    def calculate(self,predictions,y):
        comparisions = self.compare(predictions,y)
        accuracy = np.mean(comparisions)
        self.accumulated_sum += accuracy
        self.accumulated_count += 1
        return accuracy
    
    def calculate_accumulated(self):
        accuracy = self.accumulated_sum / self.accumulated_count
        return accuracy
    
    def new_pass(self):
        self.accumulated_count = 0
        self.accumulated_sum = 0
    
class Accuracy_Regression(Accuracy):
    def __init__(self):
        self.precision = None
        
    def init(self, y, reinit=False):
        if self.precision is None or reinit:
            self.precision = np.std(y) / 250

    def compare(self, predictions, y):
        return np.absolute(predictions - y) < self.precision

class Accuracy_Classification(Accuracy):
    def init(self,y):
        pass
    def compare(self,predictions,y):
        if len(y.shape) == 2:
            y = np.argmax(y,axis=1)
        return np.mean(predictions == y)

In [96]:
model = Model()
model.add(Dense_Layer(784,64))
model.add(Activation_Relu())
model.add(Dense_Layer(64,64))
# model.add(Activation_Relu())
# model.add(Dense_Layer(64,64))
model.add(Activation_Relu())
model.add(Dense_Layer(64,2))
model.add(Activation_Softmax())
model.set(loss=CrossEntropyLoss(),accuracy=Accuracy_Classification(),optimizer=Adam_Optimizer(decay_rate=1e-5))

model.finalize()

model.train(X_train,y_train,epochs=10,print_every=10,validation_data=(X_test,y_test))

Epoch :  1
iteration: 0, acc: 0.028, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 0.028, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
Epoch :  2
iteration: 0, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
Epoch :  3
iteration: 0, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
Epoch :  4
iteration: 0, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
Epoch :  5
iteration: 0, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
Epoch :  6
iteration: 0, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.000), lr: 0.001
training, acc: 1.000, loss: inf (data_loss: inf, reg_loss: 0.00

C:\Users\omalv\AppData\Local\Temp\ipykernel_10484\1884628948.py:150: RuntimeWarning: divide by zero encountered in log
  negative_log_likelihoods = -np.log(correct_confidences)


In [81]:
pred = model.forward(X_test,training=False)

In [82]:
pred =  np.argmax(pred,axis=1)

In [93]:
np.mean(pred == y_test)

0.5285714285714286